# VERSPM Model Interface

In [1]:
import emat
import os
import pandas as pd
import numpy as np
import gzip
from emat.util.show_dir import show_dir, show_file_contents

This notebook is meant to illustrate the use of TMIP-EMAT's
with VisionEval's RSPM Model.  It provides an illustration of how to use 
TMIP-EMAT and the interface to run the model.

In this example notebook, we will activate some logging features.  The 
same logging utility is written directly into the EMAT and the
`emat_verspm.py` module. This will give us a view of what's happening
inside the code as it runs.

In [2]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `emat_verspm.py`
module, which we will import into this notebook. 

In [3]:
import emat_verspm

Let's initialize a database file to store results.

In [4]:
database_path = os.path.expanduser("~/EMAT-VE/ve-rspm-20200728.db")
initialize = not os.path.exists(database_path)
db = emat.SQLiteDB(database_path, initialize=initialize)

[00:14.38] MainProcess/INFO: running script scope.sql
[00:14.38] MainProcess/INFO: running script exp_design.sql
[00:14.39] MainProcess/INFO: running script meta_model.sql


Within this module, you will find a definition for the 
`VERSPModel` class.  We initialize an instance of the model interface object.

In [5]:
fx = emat_verspm.VERSPModel(db=db)

[00:14.40] MainProcess/WARNING: changing cwd to /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1


We'll run a reference experiment with all default values to establish a baseline set of results.

In [6]:
fx.run_reference_experiment()

[00:15.83] MainProcess/INFO: performing 1 scenarios/policies * 1 model(s) = 1 experiments
[00:15.85] MainProcess/INFO: performing experiments sequentially
[00:15.85] MainProcess/INFO: VERSPM SETUP...
[00:15.96] MainProcess/INFO: VERSPM SETUP complete
[00:15.96] MainProcess/INFO: VERSPM RUN ...
[02:34.77] MainProcess/INFO: VERSPM RUN complete
[02:35.39] MainProcess/INFO: VERSPM ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1/VERSPM/output
   to: /Users/jeffnewman/sandbox/VisionEval-Archive/scp_VERSPM/exp_1.zip
[02:39.44] MainProcess/INFO: 1 cases completed
[02:39.45] MainProcess/INFO: experiments finished


,ValueOfTime,Income,LandUse,FuelCost,ElectricCost,TechMix,VehicleCharacteristics,VehicleTravelCost,Bicycles,Transit,...,DemandManagement,DrivingEfficiency,GHGReduction,DVMTPerCapita,WalkTravelPerCapita,AirPollutionEm,FuelUse,TruckDelay,VehicleCost,VehicleCostLow
experiment,,,,,,,,,,,,,,,,,,,,,
1,16.0,46367,base,2.43,0.08,0.0,0.0,base,0.08,1.0,...,0.0,0.0,0.0,20.564719,0.342725,855480.053614,3.755158e+07,0.0,3.985622,21.143855


## Single Run Operation for Development and Debugging

Before we take on the task of running this model in exploratory mode, we'll
want to make sure that our interface code is working correctly. To check each
of the components of the interface (setup, run, post-process, load-measures,
and archive), we can run each individually in sequence, and inspect the results
to make sure they are correct.

### setup

This method is the place where the core model *set up* takes place,
including creating or modifying files as necessary to prepare
for a core model run.  When running experiments, this method
is called once for each core model experiment, where each experiment
is defined by a set of particular values for both the exogenous
uncertainties and the policy levers.  These values are passed to
the experiment only here, and not in the `run` method itself.
This facilitates debugging, as the `setup` method can be used 
without the `run` method, as we do here. This allows us to manually
inspect the prepared files and ensure they are correct before
actually running a potentially expensive model.

Each input exogenous uncertainty or policy lever can potentially
be used to manipulate multiple different aspects of the underlying
core model.  For example, a policy lever that includes a number of
discrete future network "build" options might trigger the replacement
of multiple related network definition files.  Or, a single uncertainty
relating to the cost of fuel might scale both a parameter linked to
the modeled per-mile cost of operating an automobile and the
modeled total cost of fuel used by transit services.

In our RSPM module's `setup`, parameters that are omitted are set at their
deafult values, but we can give a subset of parameters with non-default values
if we like.

In [7]:
params = {
    'ValueOfTime': 13,
    'Income': 46300,
    'Transit': 1.34,
    'ElectricCost': 0.14,
    'FuelCost': 4.25,
} 

fx.setup(params)

[04:42.73] MainProcess/INFO: VERSPM SETUP...
[04:42.73] MainProcess/WARNING:  - for LandUse using default value base
[04:42.73] MainProcess/WARNING:  - for TechMix using default value 0.0
[04:42.74] MainProcess/WARNING:  - for VehicleCharacteristics using default value 0.0
[04:42.74] MainProcess/WARNING:  - for VehicleTravelCost using default value base
[04:42.74] MainProcess/WARNING:  - for Bicycles using default value 0.08
[04:42.74] MainProcess/WARNING:  - for Parking using default value 0.0
[04:42.74] MainProcess/WARNING:  - for DemandManagement using default value 0.0
[04:42.74] MainProcess/WARNING:  - for DrivingEfficiency using default value 0.0
[04:42.84] MainProcess/INFO: VERSPM SETUP complete


### run

The `run` method is the place where the core model run takes place.
Note that this method takes no arguments; all the input
exogenous uncertainties and policy levers are delivered to the
core model in the `setup` method, which will be executed prior
to calling this method. This facilitates debugging, as the `setup`
method can be used without the `run` method as we did above, allowing
us to manually inspect the prepared files and ensure they
are correct before actually running a potentially expensive model.

In [8]:
fx.run()

[05:11.76] MainProcess/INFO: VERSPM RUN ...
[07:32.30] MainProcess/INFO: VERSPM RUN complete


The `VERSPModel` class includes a custom `last_run_logs` method,
which displays both the "stdout" and "stderr" logs generated by the 
model executable during the most recent call to the `run` method.
We can use this method for debugging purposes, to identify why the 
core model crashes (if it does crash).  In this first test it did not
crash, and the logs look good.

In [9]:
show_dir(os.path.join(fx.master_directory.name, 'VERSPM', 'output'))

output/
├── Azone.csv
├── Bzone.csv
├── Household.csv
├── Marea.csv
├── Region.csv
├── Vehicle.csv
└── Worker.csv


In [10]:
fx.last_run_logs()

=== STDOUT ===
run_model.R: script entered
run_model.R: library visioneval loaded
[1] "2020-07-28 17:25:24 -- Initializing Model. This may take a while."
[1] "2020-07-28 17:25:29 -- Model successfully initialized."
run_model.R: initializeModel completed
[1] "2020-07-28 17:25:29 -- Starting module 'CreateHouseholds' for year '2010'."
[1] "2020-07-28 17:25:31 -- Finish module 'CreateHouseholds' for year '2010'."
[1] "2020-07-28 17:25:31 -- Starting module 'PredictWorkers' for year '2010'."
[1] "2020-07-28 17:25:33 -- Finish module 'PredictWorkers' for year '2010'."
[1] "2020-07-28 17:25:33 -- Starting module 'AssignLifeCycle' for year '2010'."
[1] "2020-07-28 17:25:34 -- Finish module 'AssignLifeCycle' for year '2010'."
[1] "2020-07-28 17:25:34 -- Starting module 'PredictIncome' for year '2010'."
[1] "2020-07-28 17:25:37 -- Finish module 'PredictIncome' for year '2010'."
[1] "2020-07-28 17:25:37 -- Starting module 'PredictHousing' for year '2010'."
[1] "2020-07-28 17:25:39 -- Finish modu

In [11]:
os.path.join(fx.master_directory.name, 'VERSPM', 'output')

'/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1/VERSPM/output'

### post-process

There is a `post_process` step that is separate from the `run` step.

For VERSPM, the post-processing replicates the calculations needed to
create the same summary performance measures as the _R_ version of
VisionEval does when run with scenarios.

In [12]:
fx.post_process()

### load-measures

The `load_measures` method is the place to actually reach into
files in the RSPM's run results and extract performance
measures, returning a dictionary of key-value pairs for the 
various performance measures.  

In [13]:
fx.load_measures()

{'GHGReduction': 0.0,
 'DVMTPerCapita': 19.931893301649605,
 'WalkTravelPerCapita': 0.3472318360421489,
 'TruckDelay': 0.0,
 'AirPollutionEm': 824329.461495448,
 'FuelUse': 36263244.806371376,
 'VehicleCost': 3.969387886906534,
 'VehicleCostLow': 20.830175577270325}

### archive

The `archive` method copies the relevant model output files to an archive location for 
longer term storage.  The particular archive location is based on the experiment id
for a particular experiment, and can be customized if desired by overloading the 
`get_experiment_archive_path` method.  This customization is not done in this demo,
so the default location is used.

In [16]:
fx.get_experiment_archive_path(parameters=params)

'/Users/jeffnewman/sandbox/VisionEval-Archive/scp_VERSPM/exp_2'

Actually running the `archive` method should copy any relevant output files
from the `model_path` of the current active model into a subdirectory of `archive_path`.

In [17]:
fx.archive(params)

[08:17.37] MainProcess/INFO: VERSPM ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1/VERSPM/output
   to: /Users/jeffnewman/sandbox/VisionEval-Archive/scp_VERSPM/exp_2.zip


It is permissible, but not required, to simply copy the entire contents of the 
former to the latter, as is done in this example. However, if the current active model
directory has a lot of boilerplate files that don't change with the inputs, or
if it becomes full of intermediate or temporary files that definitely will never
be used to compute performance measures, it can be advisable to selectively copy
only relevant files. In that case, those files and whatever related sub-directory
tree structure exists in the current active model should be replicated within the
experiments archive directory.

## Single Thread Operation for Running Multiple Experiments

For this demo, we'll create a design of experiments with only 3 experiments.
The `design_experiments` method of the `VERSPModel` object is not defined
in the custom code written for this model, but rather is a generic
function provide by the TMIP-EMAT main library.
Real applications will typically use a larger number of experiments, but this small number
is sufficient to demonstrate the operation of the tools.

In [19]:
design1 = fx.design_experiments(n_samples=3)
design1

,Bicycles,DemandManagement,DrivingEfficiency,ElectricCost,FuelCost,Income,LandUse,Parking,TechMix,Transit,ValueOfTime,VehicleCharacteristics,VehicleTravelCost
experiment,,,,,,,,,,,,,
3,0.097509,0.732840,0.337636,0.138558,5.114339,47616,base,0.600716,0.072931,2.903111,15.911097,0.109889,pay-per-mile insurance and higher cost
4,0.151414,0.271721,0.162278,0.089198,2.424062,46152,growth,0.047922,0.974956,0.715002,13.800396,0.500989,steady ownership cost
5,0.191199,0.386272,0.777005,0.164782,6.818868,44249,growth,0.901420,0.480714,1.721914,16.705259,0.703965,base


The `run_experiments` command will automatically run the model once for each experiment in the named design.

In [21]:
fx.run_experiments(design1)

[10:14.95] MainProcess/INFO: performing 3 scenarios/policies * 1 model(s) = 3 experiments
[10:14.97] MainProcess/INFO: performing experiments sequentially
[10:14.97] MainProcess/INFO: VERSPM SETUP...
[10:15.09] MainProcess/INFO: VERSPM SETUP complete
[10:15.09] MainProcess/INFO: VERSPM RUN ...
[12:40.45] MainProcess/INFO: VERSPM RUN complete
[12:41.06] MainProcess/INFO: VERSPM ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1/VERSPM/output
   to: /Users/jeffnewman/sandbox/VisionEval-Archive/scp_VERSPM/exp_3.zip
[12:45.09] MainProcess/INFO: 1 cases completed
[12:45.10] MainProcess/INFO: VERSPM SETUP...
[12:45.20] MainProcess/INFO: VERSPM SETUP complete
[12:45.20] MainProcess/INFO: VERSPM RUN ...
[15:07.54] MainProcess/INFO: VERSPM RUN complete
[15:08.18] MainProcess/INFO: VERSPM ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpkr469dj1/VERSPM/output
   to: /Users/jeffnewman/sandbox/VisionEval-Archive/scp_VERSPM/exp_4.zip
[15:12.30] MainProcess

,ValueOfTime,Income,LandUse,FuelCost,ElectricCost,TechMix,VehicleCharacteristics,VehicleTravelCost,Bicycles,Transit,...,DemandManagement,DrivingEfficiency,GHGReduction,DVMTPerCapita,WalkTravelPerCapita,AirPollutionEm,FuelUse,TruckDelay,VehicleCost,VehicleCostLow
experiment,,,,,,,,,,,,,,,,,,,,,
3,15.911097,47616,base,5.114339,0.138558,0.072931,0.109889,pay-per-mile insurance and higher cost,0.097509,2.903111,...,0.732840,0.337636,0.0,18.000596,0.366869,735356.850619,3.254564e+07,0.0,4.127657,20.930621
4,13.800396,46152,growth,2.424062,0.089198,0.974956,0.500989,steady ownership cost,0.151414,0.715002,...,0.271721,0.162278,0.0,19.109837,0.343564,799540.311867,3.514125e+07,0.0,4.125796,24.288610
5,16.705259,44249,growth,6.818868,0.164782,0.480714,0.703965,base,0.191199,1.721914,...,0.386272,0.777005,0.0,16.595456,0.360290,682996.353222,3.037267e+07,0.0,4.260763,23.091208


## Multiprocessing for Running Multiple Experiments

The examples above are all single-process demonstrations of using TMIP-EMAT to run core model
VERSPM experiments. 

This core model is single threaded, and such that you can run multiple independent instances of
the model side-by-side on the same machine, so you can benefit from a multiprocessing 
approach.  This can be accomplished by splitting a design of experiments over several
processes that you start manually, or by using an automatic multiprocessing library such as 
`dask.distributed`.

In [20]:
design3 = fx.design_experiments(random_seed=3, sampler='ulhs')
design3

,Bicycles,DemandManagement,DrivingEfficiency,ElectricCost,FuelCost,Income,LandUse,Parking,TechMix,Transit,ValueOfTime,VehicleCharacteristics,VehicleTravelCost
experiment,,,,,,,,,,,,,
6,0.119354,0.439522,0.742244,0.144676,5.728645,46109,growth,0.895418,0.650310,0.781931,14.545257,0.377544,steady ownership cost
7,0.156030,0.207709,0.406488,0.225597,9.619683,47636,base,0.824396,0.358715,1.420348,17.808696,0.532672,pay-per-mile insurance and higher cost
8,0.183895,0.447767,0.711325,0.136236,3.815650,44209,base,0.516359,0.757043,3.424773,17.232939,0.250093,steady ownership cost
9,0.179874,0.347820,0.317658,0.197505,4.445836,44033,growth,0.348046,0.044185,0.960535,11.237197,0.292991,steady ownership cost
10,0.094023,0.365510,0.563651,0.218112,5.935164,44966,growth,0.806866,0.521360,2.906226,13.888446,0.757025,pay-per-mile insurance and higher cost
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.097688,0.761365,0.238393,0.137463,5.573693,48504,base,0.264968,0.032807,0.922819,18.897466,0.664787,pay-per-mile insurance and higher cost
132,0.196012,0.081720,0.382707,0.125348,6.819378,48362,growth,0.758368,0.023555,1.978107,19.703536,0.921498,pay-per-mile insurance and higher cost
133,0.100119,0.870322,0.133027,0.084278,2.608404,45470,growth,0.812165,0.843943,1.090906,14.826562,0.840248,base


The module is set up to facilitate distributed multiprocessing. During the `setup`
step, the code detects if it is being run in a distributed "worker" environment instead of
in a normal Python environment.  If the "worker" environment is detected, then a copy
of the entire VERSPM model is made into the worker's local workspace, and the model
is run there instead of in the master workspace.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.

To run the model with parallel subprocesses,
we simply import the `get_client` function, and use that for the `evaluator` argument
in the `run_experiments` method.

In [22]:
from emat.util.distributed import get_client # for multi-process operation

In [23]:
fx.run_experiments(design=design3, evaluator=get_client(n_workers=8))

[17:43.47] MainProcess/INFO: performing 130 scenarios/policies * 1 model(s) = 130 experiments
[24:53.78] MainProcess/INFO: 13 cases completed
[31:39.62] MainProcess/INFO: 26 cases completed
[38:25.95] MainProcess/INFO: 39 cases completed
[45:10.83] MainProcess/INFO: 52 cases completed
[51:48.78] MainProcess/INFO: 65 cases completed
[52:05.43] MainProcess/INFO: 78 cases completed
[58:45.86] MainProcess/INFO: 91 cases completed
[01:05:20.36] MainProcess/INFO: 104 cases completed
[01:11:54.33] MainProcess/INFO: 117 cases completed
[01:16:38.40] MainProcess/INFO: 130 cases completed
[01:16:38.40] MainProcess/INFO: experiments finished


,ValueOfTime,Income,LandUse,FuelCost,ElectricCost,TechMix,VehicleCharacteristics,VehicleTravelCost,Bicycles,Transit,...,DemandManagement,DrivingEfficiency,GHGReduction,DVMTPerCapita,WalkTravelPerCapita,AirPollutionEm,FuelUse,TruckDelay,VehicleCost,VehicleCostLow
experiment,,,,,,,,,,,,,,,,,,,,,
6,14.545257,46109,growth,5.728645,0.144676,0.650310,0.377544,steady ownership cost,0.119354,0.781931,...,0.439522,0.742244,0.0,17.292738,0.354209,720493.252931,3.190253e+07,0.0,4.097128,24.183181
7,17.808696,47636,base,9.619683,0.225597,0.358715,0.532672,pay-per-mile insurance and higher cost,0.156030,1.420348,...,0.207709,0.406488,0.0,16.846406,0.366690,668651.961832,2.972039e+07,0.0,4.396516,23.207600
8,17.232939,44209,base,3.815650,0.136236,0.757043,0.250093,steady ownership cost,0.183895,3.424773,...,0.447767,0.711325,0.0,18.462275,0.361775,753073.090217,3.316841e+07,0.0,4.165156,19.445694
9,11.237197,44033,growth,4.445836,0.197505,0.044185,0.292991,steady ownership cost,0.179874,0.960535,...,0.347820,0.317658,0.0,18.124322,0.346927,763649.887035,3.381746e+07,0.0,4.267022,23.692851
10,13.888446,44966,growth,5.935164,0.218112,0.521360,0.757025,pay-per-mile insurance and higher cost,0.094023,2.906226,...,0.365510,0.563651,0.0,16.424536,0.368102,674532.189552,3.002414e+07,0.0,4.506740,23.523897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,18.897466,48504,base,5.573693,0.137463,0.032807,0.664787,pay-per-mile insurance and higher cost,0.097688,0.922819,...,0.761365,0.238393,0.0,18.755298,0.354578,749381.766192,3.312727e+07,0.0,4.387489,23.822004
132,19.703536,48362,growth,6.819378,0.125348,0.023555,0.921498,pay-per-mile insurance and higher cost,0.196012,1.978107,...,0.081720,0.382707,0.0,16.458918,0.368296,672027.808238,2.999260e+07,0.0,4.287785,25.307672
133,14.826562,45470,growth,2.608404,0.084278,0.843943,0.840248,base,0.100119,1.090906,...,0.870322,0.133027,0.0,18.391247,0.348689,759550.459607,3.349309e+07,0.0,4.229506,24.356029
